# Credits to

[1] https://www.kaggle.com/mayer79/m5-forecast-keras-with-categorical-embeddings-v2 

[2] https://www.kaggle.com/ragnar123/very-fst-model

[3] https://www.kaggle.com/mayer79/m5-forecast-poisson-loss

# Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gc
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
import time
import pickle
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_poisson_deviance

# Reduce Memory Usage 

Run this function on dataframes to reduce the memory usage.

In [ ]:
'''
Got from: [1]
'''
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Create file to store encoders

In [ ]:
encoder_file = open('encoders', 'wb')

# Prepare Dataframes

In [ ]:
'''
Got from: [1]
'''
path = "../input/m5-forecasting-accuracy"

calendar = pd.read_csv(os.path.join(path, "calendar.csv"))
selling_prices = pd.read_csv(os.path.join(path, "sell_prices.csv"))
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sales = pd.read_csv(os.path.join(path, "sales_train_validation.csv"))

In [ ]:
'''
Got from: [1]
- The columns "Date" and "Weekday" are dropped as they contain redundant information.
- Normally, the column "d" is like "d_1,d_2,...". Make it "1,2,.." and the type integer
- If there is no event (I think), there is NA. We will replace them with "NoEvent" string. Originally, it was replaced with
  "missing", but I don't think it makes sense as I don't think there is missing information, I think they just left 
  the days without any event as NA.
- We enumerate most of the columns:
    - We do not enumerate "d" and "wm_yr_weak" because we will use these columns for joins.
    - Why do we enumerate month and day? I think it is because they start from 1, not 0.
    - Originally, the binary columns "snap_X" were also enumerated. I don't think it is necessary. The only neccessary step
      was to convert their type from int64 to int as it uses less space; but reduce_mem_usage will take care of that.
- I would suggest saving the OrdinalEncoder in case we need to reverse the transformations
'''
def prep_calendar(df,encoder_file):
    df = df.drop(["date", "weekday","event_name_1", "event_name_2", "event_type_2","event_type_1","snap_CA","snap_TX","snap_WI"], axis=1)  
    df = df.assign(d = df.d.str[2:].astype(int))
    df = df.fillna("NoEvent")
    cols = list(set(df.columns) - {"wm_yr_wk", "d"}) 
    oe = OrdinalEncoder(dtype="int")
    df[cols] = oe.fit_transform(df[cols])
    pickle.dump(oe,encoder_file)
    df = reduce_mem_usage(df)
    return df

calendar = prep_calendar(calendar,encoder_file)

In [ ]:
'''
Got from: [1]
Originally, there were features added in this part. I excluded them until we decide whether to use those or not.
'''
def prep_selling_prices(df):
    df = reduce_mem_usage(df)
    return df

selling_prices = prep_selling_prices(selling_prices)

In [ ]:
'''
Got from: [1]
- We drop the first "drop_d" days. Originally, this is set to 1000. When it is set to this value,
  the shape we get 29,544,810 rows. When we don't set it, we get 60,034,810 rows. I think for now 
  we can keep this functionality, as it may be useful if we would like to discard some of the days.
- In some id's, we have "_validation". Those are deleted.
- reindex: Conform DataFrame to new index with optional filling logic (obtained from pandas doc). 
  We add days 1914+2*28 to prepare data from submission
- We have to melt the sales dataframe since days are contained as columns.
- assign: Returns a new object with all original columns in addition to new ones. Existing columns 
  that are re-assigned will be overwritten (obtained from pandas doc). Again, we make the values 
  "d_1, d-2,..." to "1,2,..."
'''
#We have to melt sales for sure because the days are columns, which is not desirable.
def reshape_sales(df, drop_d = None):
    if drop_d is not None:
        df = df.drop(["d_" + str(i + 1) for i in range(drop_d)], axis=1)
    df = df.assign(id=df.id.str.replace("_validation", ""))
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(1913 + i + 1) for i in range(2 * 28)])
    df = df.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                 var_name='d', value_name='demand')
    df = df.assign(d=df.d.str[2:].astype("int16"))
    return df

sales = reshape_sales(sales)

# Merge Dataframes

In [ ]:
'''
Got from: [1]
- Merge all the dataframes and delete the unnecessary ones
- time.sleep() added to make sure garbage collector finishes its job before the next merge
'''
sales = sales.merge(calendar, how="left", on="d")
del calendar
gc.collect()
time.sleep(5)
sales = sales.merge(selling_prices, how="left", on=["wm_yr_wk", "store_id", "item_id"])
del selling_prices
sales.drop(["wm_yr_wk"], axis=1, inplace=True)
gc.collect()
time.sleep(5)
sales.head()

# Encode some variables

In [ ]:
'''
Got from: [1]
- We will also save the encoders in the pickle file.
- The loop is slightly changed
'''

cat_id_cols = ["item_id", "dept_id", "store_id", "cat_id", "state_id"]

# In loop to minimize memory use
for col in cat_id_cols:
    oe = OrdinalEncoder(dtype="int")
    sales[col] = oe.fit_transform(sales[[col]])
    pickle.dump(oe,encoder_file)    
sales = reduce_mem_usage(sales)
gc.collect()

In [ ]:
# imputing NaN of demand with 0 

sales["demand"] = sales["demand"].fillna(0)

# Feature Engineering

When we do train,test, validation split; for some time we have to store all data twice to be able to divide the dataset. And every feature we add increases the size of the dataset drastically. Hence, if you have memory issues, I would suggest doing this part after splitting, even if it means we have to execute each line 3 times (for train,test,valid). And we have to be careful for the first few rows because these features sometimes depend on previous datapoints; which may be found in train set for first few rows of validation, for example.

KNN also may require some extra data storage. Moreover, in that case, this part can be even after KNN. This way, you can also use imputed selling prices to calculate your new features.

In [ ]:
'''
Got from: [1]
- These features were originally added in prep_selling_prices function to the 
  selling_prices dataframe, which does not exist anymore. But we can use the same
  code to add these to sales dataframe as well as the columns are the same.
- New feature="sell_price_rel_diff"
  pct_change(): Computes the percentage change from the immediately previous row by default. (Obtained from pandas doc.)
  The two lines below adds the percentage of change of each item that is sold in the stores. Of course, for the
  first datapoint, there is no previous, so this code produces an NA. Since there are 3049x10=30490 different (item,store)
  pairs, this new column has 30490 NAs.
- New feature="sell_price_roll_sd7"
  Rolling standard deviation: Moving Standard Deviation is a statistical measurement of market volatility (Google). We check the
  past 7 days.
- New feature="sell_price_cumrel"
  I think this is cumulative related frequency. I am not sure and I did not understand it clearly. I think
  it is some kind of normalization, because we subtract the minimum and divide by max-min+1.
- It runs without problems, but the RAM gets almost filled up so it gives you a heart attack.
- No need to call reduce_mem_usage() after as I tried and it did not save any additional space.
'''

#gr = sales.groupby(["store_id", "item_id"])["sell_price"]
#sales["sell_price_rel_diff"] = gr.pct_change()

#del gr
#gc.collect()



In [ ]:
import scipy;
from scipy import stats
'''
Got from: [1]
Appearantly, [1] derived the features from [2].

- The features include rolling means and standard deviations for different number of days.
  I think this is also a measure of market volatiliy.
- After this, original notebook deletes rows producing NAs. 'rolling_mean_t180' produces the most NAs as I
  believe the value of first 180 days will be NA because to compute this we need 180 days prior. I do not 
  know whether adding these features is so important to delete that much data. Hence, I will keep it for now.
  I think time ranges can also be changed.
'''

gr = sales.groupby(["id"])["demand"]

# fourier 
fourier = gr.transform(lambda x: np.fft.ifft(x))
sales['ifourier_lag_t14'],sales['ifourier_lag_t28'] = fourier,fourier
print(sales.head(10))
sales['ifourier_lag_t14'] = sales['ifourier_lag_t14'].transform(lambda x: x.shift(14))
sales['ifourier_lag_t28'] = sales['ifourier_lag_t28'].transform(lambda x: x.shift(28))

fourier = gr.transform(lambda x: scipy.fft.idct(x , type=2, norm='ortho'))
sales['idfourier_lag_t14'], sales['idfourier_lag_t28'] = fourier, fourier
print(sales.head(10))
sales['idfourier_lag_t14'] = sales['idfourier_lag_t14'].transform(lambda x: x.shift(14)) 
sales['idfourier_lag_t28'] = sales["idfourier_lag_t28"].transform(lambda x: x.shift(28))



del gr, fourier
sales = reduce_mem_usage(sales)
gc.collect()

# commented this just so see impact of fourier 

'''sales['rolling_mean_t7'] = gr.transform(lambda x: x.shift(28).rolling(7).mean())
sales['rolling_mean_t30'] = gr.transform(lambda x: x.shift(28).rolling(30).mean())
sales['rolling_mean_t60'] = gr.transform(lambda x: x.shift(28).rolling(60).mean())
sales['rolling_mean_t180'] = gr.transform(lambda x: x.shift(28).rolling(180).mean())
#sales['rolling_std_t7'] = gr.transform(lambda x: x.shift(28).rolling(7).std())
sales['rolling_std_t30'] = gr.transform(lambda x: x.shift(28).rolling(30).std())
'''

# lora's custom features
'''
sales['price_max'] = gr.transform('max')
sales['price_min'] = gr.transform('min')
sales['price_std'] = gr.transform('std')
sales['price_mean'] =gr.transform('mean')
sales['price_norm'] = sales['sell_price']/sales['price_max']
sales['price_momentum'] =   sales['sell_price']/gr.transform(lambda x: x.shift(1))
sales['price_momentum_m'] = sales['sell_price']/gr.transform('mean')
sales['price_momentum_y'] = sales['sell_price']/gr.transform('mean')
'''


In [ ]:

del  fourier
sales = reduce_mem_usage(sales)
gc.collect()


In [ ]:
sales.head(50)

# Train Test Split

Naming convention:
* test, train, valid -> Use these to create your X_{train,valid,test}  and y_{train,valid}.
* Training data: X_train, y_train
* Validation data: X_valid, y_valid
* Testing data (Kaggle has the labels): X_test

X_{train,valid,test} should be preprocessed according to your model.

Since we will use test dataframe for submission and the submission is obtained from another person, make sure the order of X_test and test is the same.

This process increases memory usage from 6GB to 7.5GB and reduce_mem_usage() does not improve the situation. One idea can be to keep the flags generated but produce end datasets at the end.

In [ ]:
'''
Got from: [1]
'''
test = sales[sales.d >= 1914]
test = test.assign(id=test.id + "_" + np.where(test.d <= 1941, "validation", "evaluation"),
                   F="F" + (test.d - 1913 - 28 * (test.d > 1941)).astype("str"))
gc.collect()

# One month of validation data
flag_valid = (sales.d < 1914) & (sales.d >= 1914 - 28)
valid = sales[flag_valid]
flag_train = sales.d < 1914 - 28
train = sales[flag_train]


del sales, flag_valid,flag_train
gc.collect()
time.sleep(5)

test.reset_index(drop=True)
valid.reset_index(drop=True)
train.reset_index(drop=True)

gc.collect()

# Impute NAs

Documentation says: If (selling_price) not available, this means that the product was not sold during the examined week. 

When we are not using recurrent structures, I think it is the
best if we just remove those rows. However, for recurrent structures, such as LSTM and GRU, it would be better if
the sequence was not broken. For those models, we can impute the missing labels with KNN. So there will be two
alternatives. Set impute_labels=True to impute, False to drop depending on your application.

Note that we have to do the imputation after we split the data to prevent leaking information.

Don't use the demand column not to leak data from the label.

I could not use KNN, SVR etc due to memory limitations.



In [ ]:
print("NAs:\n",train.isna().sum())
impute_labels = True

if not impute_labels:
    train.dropna(inplace=True)
    valid.dropna(inplace=True)
    gc.collect()
    time.sleep(2)
else:
    to_be_inputed = ["ifourier_lag_t14","ifourier_lag_t28",
                    'idfourier_lag_t14','idfourier_lag_t28']
    for col in to_be_inputed:
        mean_train = train[col].dropna().astype(float).mean(skipna=True)
        print(mean_train)
        train[col] = train[col].fillna(mean_train)
        valid[col] = valid[col].fillna(mean_train)
        test[col] = test[col].fillna(mean_train)
gc.collect()


# Normalization

We have to normalize data for the models that are not tree based. 

In [ ]:
x = list(set(train.columns) - {'id', 'd', 'demand'})
train_part = train[train['year'] > 2]
del train

# Close encoder file

In [ ]:
encoder_file.close()

# Model

** Enter description of the model here **

** Name your model "model" **

In [ ]:
train_set = lgb.Dataset(train_part.drop(['id', 'demand'], axis=1), label = train_part['demand'],
                        categorical_feature=['item_id','dept_id','cat_id','store_id','state_id','wday',
                                             'month','year'])
del train_part
valid_set = lgb.Dataset(valid.drop(['id', 'demand'],axis=1), label = valid['demand'],
                        categorical_feature=['item_id','dept_id','cat_id','store_id','state_id','wday',
                                             'month','year'])
del valid

In [ ]:
# taken from [3]
def fit_model(train, valid):
    """ Fit LightGBM model """
     
    params = {
        'metric': 'rmse',
        'objective': 'poisson',
        'seed': 200,
        'force_row_wise' : True,
        'learning_rate' : 0.075,
        #'lambda': 0.1,
        'num_leaves': 128,
        'sub_row' : 0.7,
        'bagging_freq' : 1,
        'colsample_bytree': 0.77,
        "lambda_l2" : 0.1,
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    "min_data_in_leaf": 100
    }

    fit = lgb.train(params, 
                    train, 
                    num_boost_round = 2000, 
                    valid_sets = [valid], 
                    early_stopping_rounds = 200,
                    verbose_eval = 100)
    
    lgb.plot_importance(fit, importance_type="gain", precision=0, height=0.5, figsize=(6, 10));
    
    return fit

fit = fit_model(train_set, valid_set)

# Submission

In [ ]:
'''
Got from: [1]
'''

pred = fit.predict(test.drop(['id','F', 'demand'],axis=1))
test["demand"] = pred.clip(0)
submission = test.pivot(index="id", columns="F", values="demand").reset_index()[sample_submission.columns]
submission = sample_submission[["id"]].merge(submission, how="left", on="id")
submission.to_csv("submission.csv", index=False)
